In [2]:
import pandas as pd
import numpy as np
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from collections import namedtuple
import matplotlib.pyplot as plt
import threading
import pygame
import random
import mido
import math
import os
import rtmidi
from sklearn.cluster import KMeans
# from collections import defaultdicta

col = pd.read_csv('/Users/diegoellis/projects/Proposals_funding/Yale_internal_grants/Franke_program/CCAM_Music_migration/Data/Pol_Rot_datasets/Pigeon_Nagy_hf_rotation_polarizaiton_group_data.csv')

# the numpy array 'allparticles' contains results stored as:
# [timestep, particle, quantity] The different quantities are: x, y, vx, vy, spd, ang, angvel, stopgo, state


# Set the domain based on the full data:

ind = pd.read_csv('/Users/diegoellis/projects/Proposals_funding/Yale_internal_grants/Franke_program/CCAM_Music_migration/Data/Pol_Rot_datasets/Pigeon_Nagy_hf_rotation_polarizaiton_full-data.csv')

ind.head()
    
domains_individuals = {'x': [
    ind['pos_x'].min(),
    ind['pos_y'].min()
], 'y': [
    ind['pos_x'].max(),
    ind['pos_y'].max()
]}

error_term = 0.01

domains_rot_pol  = {'Polarization': [
    col['Polarization'].min() - error_term,
    col['Polarization'].max() + error_term
], 'Rotation': [
    col['Rotation'].min() - error_term,
    col['Rotation'].max() + error_term
]}


# col.head()


domains = domains_rot_pol
# coords = np.array(ind['pos_x', 'pos_y'])

# coords


pygame 2.1.2 (SDL 2.0.18, Python 3.10.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Look at the variables in head:

col.head()
col.tail()
ind.head()

Unnamed: 0     id    frame   pos_x   pos_y  plotting     dir_x     dir_y  \
0           1  hf1_A  1745420 -0.7403  0.2016     False  0.897745  0.440515   
1           2  hf1_A  1745440 -2.1696 -0.2593     False  0.881132  0.472870   
2           3  hf1_A  1745460 -2.9650  0.0033     False  0.835145  0.550030   
3           4  hf1_A  1745480 -3.2572  0.8514     False  0.712607  0.701564   
4           5  hf1_A  1745500 -3.1710  1.9035     False  0.080497  0.996755   

      speed  centroid_x  centroid_y  group_dir_x  group_dir_y  group_speed  \
0  7.890991   -0.071550    0.011525     0.064377     0.142890     1.628662   
1  7.254418   -0.267450   -0.050362     0.044912     0.174265     1.500440   
2  7.287239   -0.353450   -0.046050     0.186959     0.142429     1.439456   
3  5.605194   -0.352625    0.022638     0.222501     0.161264     1.191121   
4  2.906933   -0.297600    0.126525     0.216131     0.171994     0.847872   

   centroid_disp_x  centroid_disp_y  rotation       t_       seconds  
0        -0.961902         0.273396 -0.669172  1745420  58180.666667  
1        -0.994021        -0.109186 -0.373836  1745440  58181.333333  
2        -0.999822         0.018893 -0.565711  1745460  58182.000000  
3        -0.961621         0.274379 -0.870163  1745480  58182.666667  
4        -0.850503         0.525970 -0.890082  1745500  58183.333333

In [3]:
ind.head
# Use the speed of all for entropy
# Use the speed for how long the notes are

# Use their speed for change in pitch frequencty.


<bound method NDFrame.head of         Unnamed: 0     id    frame   pos_x   pos_y  plotting     dir_x  \
0                1  hf1_A  1745420 -0.7403  0.2016     False  0.897745   
1                2  hf1_A  1745440 -2.1696 -0.2593     False  0.881132   
2                3  hf1_A  1745460 -2.9650  0.0033     False  0.835145   
3                4  hf1_A  1745480 -3.2572  0.8514     False  0.712607   
4                5  hf1_A  1745500 -3.1710  1.9035     False  0.080497   
...            ...    ...      ...     ...     ...       ...       ...   
156629      156630  hf1_L  2055980 -1.0479  1.1060     False -0.358295   
156630      156631  hf1_L  2056000 -0.8772  0.9795     False  0.046018   
156631      156632  hf1_L  2056020 -0.7029  0.8264     False  0.951125   
156632      156633  hf1_L  2056040 -0.5421  0.6607     False  0.936061   
156633      156634  hf1_L  2056060 -0.4670  0.5773     False  0.861401   

           dir_y     speed  centroid_x  centroid_y  group_dir_x  group_dir_y  \
0

In [5]:
# coords = np.array(ind['pos_x', 'pos_y'])
# Convert individual location data into numpy
ind.head()

coords = ind[['pos_x', 'pos_y']]


coords_array = coords.to_numpy()

domains

{'Polarization': [-0.0093321588736607, 1.009999997571752],
 'Rotation': [-1.0097622039395069, 1.0006203016424629]}

In [5]:
# ind.t_.unique().shape
# ind.group_speed.tail()

In [4]:

# Lets try rotaiton and polarizaiton on a pitch grid

import random
import numpy as np

# identify available pitch steps within chromatic scale
'''
c  0
c# 1
d  2
d# 3
e  4
f  5
f# 6
g  7
g# 8
a  9
a# 10
b  11
'''

# base_steps = [0, 4, 7]

base_steps = [0, 2, 4, 5, 7, 9, 11]


steps = []
for octave in [3,4,5]:
  steps += [i+(octave*12) for i in base_steps]

# identify the number of distinct pitches in each dimension
n_x_stops = 10
n_y_stops = 10

# quantize out a grid
x_stops = np.linspace(domains['Rotation'][0], domains['Rotation'][1], num=n_x_stops, endpoint=False)
y_stops = np.linspace(domains['Polarization'][0], domains['Polarization'][1], num=n_y_stops, endpoint=False)

# create the coordinate grid
coord_grid = np.array(np.meshgrid(x_stops, y_stops)).T
# 

# We can replace the pitch grid with a multidimensional tonnetz where we use the 
# 

# create the grid of pitch steps
pitch_grid = np.zeros((n_x_stops, n_y_stops))
kdx = 0
for idx, i in enumerate(pitch_grid):
    for jdx, j in enumerate(i):
        pitch_grid[idx, jdx] = steps[kdx]
        kdx = kdx+1
        if kdx >= len(steps): kdx = 0
pitch_grid = pitch_grid.astype('int')

In [6]:
# steps indicates the full list of notes that can be played in this pitch grid
print(steps)

[36, 38, 40, 41, 43, 45, 47, 48, 50, 52, 53, 55, 57, 59, 60, 62, 64, 65, 67, 69, 71]


In [7]:
# display the full pitch grid
pitch_grid

array([[36, 38, 40, 41, 43, 45, 47, 48, 50, 52],
       [53, 55, 57, 59, 60, 62, 64, 65, 67, 69],
       [71, 36, 38, 40, 41, 43, 45, 47, 48, 50],
       [52, 53, 55, 57, 59, 60, 62, 64, 65, 67],
       [69, 71, 36, 38, 40, 41, 43, 45, 47, 48],
       [50, 52, 53, 55, 57, 59, 60, 62, 64, 65],
       [67, 69, 71, 36, 38, 40, 41, 43, 45, 47],
       [48, 50, 52, 53, 55, 57, 59, 60, 62, 64],
       [65, 67, 69, 71, 36, 38, 40, 41, 43, 45],
       [47, 48, 50, 52, 53, 55, 57, 59, 60, 62]])

AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [8]:
x_stops


array([-1.0097622 , -0.80872395, -0.6076857 , -0.40664745, -0.2056092 ,
       -0.00457095,  0.1964673 ,  0.39750555,  0.5985438 ,  0.79958205])

In [9]:
# row.Rotation

In [10]:
col.head()

Unnamed: 0         t_  Rotation  Polarization             state  \
0           1  1713460.0 -0.148808      0.210614  Transition_state   
1           2  1713480.0 -0.435838      0.577666  Transition_state   
2           3  1713500.0 -0.475287      0.767618       Polar_state   
3           4  1713520.0 -0.449856      0.833466       Polar_state   
4           5  1713540.0 -0.416673      0.868458       Polar_state   

        seconds  
0  57115.333333  
1  57116.000000  
2  57116.666667  
3  57117.333333  
4  57118.000000

In [11]:
ind.iloc[2]

Unnamed: 0                3
id                    hf1_A
frame               1745460
pos_x                -2.965
pos_y                0.0033
plotting              False
dir_x              0.835145
dir_y               0.55003
speed              7.287239
centroid_x         -0.35345
centroid_y         -0.04605
group_dir_x        0.186959
group_dir_y        0.142429
group_speed        1.439456
centroid_disp_x   -0.999822
centroid_disp_y    0.018893
rotation          -0.565711
t_                  1745460
seconds             58182.0
Name: 2, dtype: object

In [12]:
ind.iloc[20]

Unnamed: 0               21
id                    hf1_A
frame               1745820
pos_x                -1.045
pos_y                0.2314
plotting              False
dir_x             -0.973412
dir_y              0.229059
speed              2.242215
centroid_x        -0.200675
centroid_y          -0.0643
group_dir_x       -0.139211
group_dir_y       -0.095979
group_speed        0.784899
centroid_disp_x   -0.943793
centroid_disp_y    0.330536
rotation           0.105563
t_                  1745820
seconds             58194.0
Name: 20, dtype: object

In [13]:
from collections import defaultdict

m = mido.MidiFile()
m.tracks.append(mido.MidiTrack())

step = 1 # how often to sample frames
time_scalar = 12 # beats per unit in i # moght not be bpm 
times = {i: 0 for i in steps} # d[note] = time the note was pressed
pressed = {i: False for i in steps} # d[note] = pressed or not
positions = []
total_time = 0

for idx, row in col.iterrows():
    
    # get the timestamp t of that index col[idx,]
    t_var = row.t_
    group_speed_var = np.mean(ind.loc[ind.t_ == t_var].group_speed)
    
    frame_notes = []
    frame_positions = []
    x_pos = np.where(x_stops < row.Rotation)[0][-1] # Compares the X rotation threshold to the rotation values and it gets the index of the last point where that condition is true
    y_pos = np.where(y_stops < row.Polarization)[0][-1]
    # this gets a xy rotaion for polarization.
    # It appends it to the pitch grid. 
    frame_notes.append( pitch_grid[x_pos, y_pos] )
    frame_positions.append([x_pos, y_pos])
    
    for note in pressed:
        if not pressed[note]: continue
        if note not in frame_notes:
            # It turns of the notes during the loop
            # - speed in note off
            
            # velocity default was 64
            # Make a varible called time
            
            m.tracks[0].append( mido.Message('note_off', note=note,velocity = int(np.round(64)), time = (idx*time_scalar))) # time= int(np.round(idx*time_scalar-8*group_speed_var))))
            pressed[note] = False
    for note in frame_notes:
        if not pressed[note]:
            pressed[note] = True
            times[note] = idx
            # plus group speed in note on 
            m.tracks[0].append( mido.Message('note_on', note=note, velocity = int(np.round(64 - group_speed_var * 2.5)), time = (idx*time_scalar)))  # time= int(np.round(idx*time_scalar+8*group_speed_var) )))
# close all open notes at the last timepoint 
for note in pressed:
    if not pressed[note]: continue
    m.tracks[0].append( mido.Message('note_off', note=note, time=idx*time_scalar) ) # Turns off all notes at the end of time

last_time = 0
for i in m.tracks[0]:
    t = i.time
    i.time = i.time - last_time

last_time = t

In [14]:
m.tracks
# In MIDI time needs to be an integer

[MidiTrack([
   Message('note_on', channel=0, note=36, velocity=60, time=0),
   Message('note_off', channel=0, note=36, velocity=64, time=12),
   Message('note_on', channel=0, note=43, velocity=59, time=12),
   Message('note_off', channel=0, note=43, velocity=64, time=24),
   Message('note_on', channel=0, note=47, velocity=59, time=24),
   Message('note_off', channel=0, note=47, velocity=64, time=36),
   Message('note_on', channel=0, note=48, velocity=58, time=36),
   Message('note_off', channel=0, note=48, velocity=64, time=60),
   Message('note_on', channel=0, note=65, velocity=59, time=60),
   Message('note_off', channel=0, note=65, velocity=64, time=72),
   Message('note_on', channel=0, note=67, velocity=60, time=72),
   Message('note_off', channel=0, note=67, velocity=64, time=120),
   Message('note_on', channel=0, note=41, velocity=61, time=120),
   Message('note_off', channel=0, note=41, velocity=64, time=132),
   Message('note_on', channel=0, note=65, velocity=60, time=132),
  

In [16]:
m.tracks[0].insert(0, mido.MetaMessage('set_tempo', tempo=990))
m.ticks_per_beat = 300
m.save('/Users/diegoellis/Desktop/nagys_pigeons_hf_3.mid')

In [17]:
os.system('open /Users/diegoellis/Desktop/nagys_pigeons_hf_3.mid')

0

In [18]:
# os.system('open /Users/diegoellis/Desktop/jons_termites_3.mid')
  
# convert to mp3
# if os.path.exists('flock.mp3'): os.remove('flock.mp3')
os.system('timidity /Users/diegoellis/Desktop/nagys_pigeons_hf_3.mid -Ow -o - | ffmpeg -i - -acodec libmp3lame -ab 64k /Users/diegoellis/Desktop/nagys_pigeons_hf_3.mp3')


Playing /Users/diegoellis/Desktop/nagys_pigeons_hf_3.mid
MIDI file: /Users/diegoellis/Desktop/nagys_pigeons_hf_3.mid
Format: 1  Tracks: 1  Divisions: 300
Maxmum number of events is exceeded
ffmpeg version 4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with clang version 11.1.0
  configuration: --prefix=/Users/diegoellis/bash/envs/discotermite --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-libvpx --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/Users/runner/miniforge3/conda-bld/ffmpeg_1636205445100/_build_env/bin/pkg-config
  libavutil      56. 70.100 / 56. 70.100
  libavcodec     58.134.100 / 58.134.100
  libavformat    58. 76.100 / 58. 76.10

0

In [17]:
# store all times in absolute coords
flock_arr = np.load('/Users/diegoellis/flocking/flock-simulation.npy')

flock_arr.shape

(10000, 50, 2)

In [ ]:
from sklearn.cluster import KMeans
from collections import defaultdict

m = mido.MidiFile()
m.tracks.append(mido.MidiTrack())

step = 1 # how often to sample frames
time_scalar = 120 # beats per unit in i
times = {i: 0 for i in steps} # d[note] = time the note was pressed
pressed = {i: False for i in steps} # d[note] = pressed or not
positions = [] # [[list of x,y positions in pitch space for frame]]


for idx, i in enumerate(flock_arr): # time
    if idx % step != 0: continue
    model = KMeans(n_clusters=3).fit(i)
    frame_centroids = model.cluster_centers_
    # convert the k flock centroid positions to notes
    frame_notes = []
    frame_positions = []
    for x, y in frame_centroids:
    # find the index offsets of x, y in the coords grid
        x_pos = np.where(x_stops < x)[0][-1]
        y_pos = np.where(y_stops < y)[0][-1]
    # get the pitch that corresponds with the centroid
        frame_notes.append( pitch_grid[x_pos, y_pos] )
        frame_positions.append([x_pos, y_pos])
        positions.append(frame_positions)
    # end each played note that's not in the new notes
    for note in pressed:
    if not pressed[note]: continue
    if note not in frame_notes:
      m.tracks[0].append( mido.Message('note_off', note=note, time=idx*time_scalar) )
      pressed[note] = False
  # add the new notes
  for note in frame_notes:
    if not pressed[note]:
      pressed[note] = True
      times[note] = idx
      m.tracks[0].append( mido.Message('note_on', note=note, time=idx*time_scalar) )
# close all open notes
for note in pressed:
  if not pressed[note]: continue
  m.tracks[0].append( mido.Message('note_off', note=note, time=idx*time_scalar) )
      
# convert times to relative offsets
last_time = 0
for i in m.tracks[0]:
  t = i.time
  i.time = i.time - last_time

last_time = t

In [ ]:
# Hasta aca llegue: 



for idx, i in enumerate(flock_arr[:n_frames]): # time
  if idx % step != 0: continue
  # we can access other variables in i (which is a single time step in our dataframe)
  speeds = i[:,4]
  if np.any(speeds < 0): print(' * got negative speeds')
  # fit kmeans on x,y (lat,lng)
  model = KMeans(n_clusters=3).fit(i[:,:2])    
  frame_centroids = model.cluster_centers_

  # find the centroid to which each animal belongs - keys are cluster indices; vals are animal indices
  cluster_dict = defaultdict(list)
  for jdx, j in enumerate(model.labels_):
    cluster_index = j
    animal_index = jdx
    cluster_dict[cluster_index].append(animal_index)
    
  # find the list of speeds for each cluster - keys are cluster indices, vals are mean speeds
  speeds_dict = {}
  for cluster_index in cluster_dict:
    animal_indices = cluster_dict[cluster_index]
    cluster_speeds = speeds[animal_indices]
    speeds_dict[cluster_index] = np.mean(cluster_speeds)

  # convert the k flock centroid positions to notes
  frame_notes = []
  frame_positions = []
  # jdx is cluster centroid, j is x,y coords of centroid
  for jdx, j in enumerate(frame_centroids):
    x, y = j
    
    # now we can pass the speed as the time scalar
    speed = abs(speeds_dict[jdx])
    # find the index offsets of x, y in the coords grid
    x_pos = np.where(x_stops < x)[0][-1]
    y_pos = np.where(y_stops < y)[0][-1]
    # get the pitch that corresponds with the centroid
    # FOr each time get xy for each centroid and append that to a pitch grid. 
    # 
    frame_notes.append( pitch_grid[x_pos, y_pos] )
    frame_positions.append([x_pos, y_pos])
  
  positions.append(frame_positions)
  # end each played note that's not in the new notes
  for note in pressed:
    if not pressed[note]: continue
    if note not in frame_notes:
      m.tracks[0].append( mido.Message('note_off', note=note, time=int(idx*speed)) )
      pressed[note] = False
  # add the new notes
  for note in frame_notes:
    if not pressed[note]:
      pressed[note] = True
      times[note] = idx
      m.tracks[0].append( mido.Message('note_on', note=note, time=int(idx*speed)) )
# close all open notes
for note in pressed:
  if not pressed[note]: continue
  m.tracks[0].append( mido.Message('note_off', note=note, time=int(idx*speed)) )
      # Midi has note on off concept, press a certain note of the keyboard at a certain time and the note off releases the key
# convert times to relative offsets
# press. a note 
last_time = 0
for i in m.tracks[0]: 
  t = i.time
  i.time = i.time - last_time

last_time = t